In [ ]:
from network import WLAN, STA_IF
from parameters import *
from time import sleep
wlan = WLAN(STA_IF)
wlan.active(True)
wlan.connect(WIFI_SSID, WIFI_PASSWORD)            
while not wlan.isconnected():
    sleep(4)


In [44]:
def limitedGetRequest(url, tagsToKeep=(), timeout=15):
    
    import ussl, usocket, gc

    scheme, _, host, target = url.split("/",3)

    if scheme == "https:":
        port = 443
    elif scheme == "http:":
        port = 80

    ai = usocket.getaddrinfo(host, port, 0, usocket.SOCK_STREAM)[0]
    s = usocket.socket(ai[0], usocket.SOCK_STREAM, ai[2])
    s.settimeout(timeout) # time for connection to timeout in seconds
    s.connect(ai[-1])
    s = ussl.wrap_socket(s, server_hostname=host)
    s.write(f"GET /{target} HTTP/1.0\r\n")
    s.write(f"Host: {host}\r\n")
    s.write(b"User-agent: Mozilla/5.0\r\n")
    s.write(b"Connection: close\r\n\r\n")
    
    keptLines = []
    strBuff = []
    buildString = False
    keepLine = False
    while True:
        
        buff = s.read(512)
        gc.collect()
        
        if buff != b"":
            for x in buff:
                
                x = chr(x)

                if buildString:
                    strBuff.append(x)
                
                if x == "<" and not buildString:
                    buildString = True
                    strBuff.append(x)
                
                elif x == ">" and buildString and not keepLine:
                    bufferTagName = "".join(strBuff[ 1: -1 ]).split(" ",1)[0]
                    
                    if any( tag in bufferTagName for tag in tagsToKeep ) and strBuff[1] != "/":
                        keepLine = True
                    else:
                        buildString = False
                        keepLine = False
                        strBuff = []
                    
                elif x == ">" and buildString and keepLine:
                    closingTag = "</" + bufferTagName + ">"
                    _ = "".join(strBuff[-len(closingTag):])
                    print(gc.mem_free(), closingTag, _)
                    if "".join(strBuff[-len(closingTag):]) == closingTag:
                        keptLines.append( "".join(strBuff) )
                        buildString = False
                        keepLine = False
                        strBuff = []
                    
        else:
            break

    s.close()
    
    return (keptLines)

from parameters import *


url = f"https://forecast.weather.gov/MapClick.php?lat={lat}&lon={long}&FcstType=digitalDWML"
print(url)
tagsToKeep = ["start-valid-time", "temperature", "probability-of-precipitation", "cloud-amount"]
print(limitedGetRequest(url, tagsToKeep, timeout=15))


"b'A'"

In [95]:
from requests import get 

response = get(f"https://alerts.weather.gov/cap/wwaatmget.php?x=IAC113&y=1")
for x in response.text.split("\n"):
    print(x)

<?xml version = '1.0' encoding = 'UTF-8' standalone = 'yes'?>
<?xml-stylesheet href='capatom.xsl' type='text/xsl'?>

<!--
This atom/xml feed is an index to active advisories, watches and warnings 
issued by the National Weather Service.  This index file is not the complete 
Common Alerting Protocol (CAP) alert message.  To obtain the complete CAP 
alert, please follow the links for each entry in this index.  Also note the 
CAP message uses a style sheet to convey the information in a human readable 
format.  Please view the source of the CAP message to see the complete data 
set.  Not all information in the CAP message is contained in this index of 
active alerts.
-->

<feed
xmlns = 'http://www.w3.org/2005/Atom'
xmlns:cap = 'urn:oasis:names:tc:emergency:cap:1.1'
xmlns:ha = 'http://www.alerting.net/namespace/index_1.0'
>
<!-- TZN = <CST> -->
<!-- TZO = <-6> -->
<!-- http-date = Sun, 28 Jan 2024 07:49:00 GMT -->
<id>https://alerts.weather.gov/cap/wwaatmget.php?x=IAC113&amp;y=0</id>
<gene

In [96]:
(x for x in "abc")

<generator object <genexpr> at 0x00000259A8649B60>

In [8]:
cautionString = ""

xmlWeatherAlerts = ["NOW", "MODERATE", "LIKELY", "NOW", "EXTREME", "LIKELY"]

if len(xmlWeatherAlerts) > 2:
    for n in range(0, len(xmlWeatherAlerts), 3):
        urgency = xmlWeatherAlerts[n].upper()
        severity = xmlWeatherAlerts[n+1].upper()
        certainty = xmlWeatherAlerts[n+2].upper()
        print(urgency, severity, certainty)
        # Possible severities are "MINOR", "MODERATE", "SEVERE", "EXTREME"
        if "UNLIKELY" not in certainty and "PAST" not in urgency:
            if severity == "MINOR" and len(cautionString) < len("!"):
                cautionString = "!"
            elif severity == "MODERATE" and len(cautionString) < len("!!"):
                cautionString = "!!"
            elif severity == "SEVERE" and len(cautionString) < len("!!!"):
                cautionString = "!!!"
            elif severity == "EXTREME" and len(cautionString) < len("*!!!!!*"):
                cautionString = "*!!!!!*"

print(cautionString)

NOW MODERATE LIKELY
NOW EXTREME LIKELY
*!!!!!*
